Takes clean and common data and compute all sort of KPIs based on offer caracteriscs (essentially NLP on descriptions). Then store the processed data in a common file which contains all the data since the beginning of the project.

- READ : *.csv - data/new_tmp_data : new_clean_data.csv
- READ : *.csv - data/processed_data : already processed data
- WRITE : *.csv - data/processed_data/history : new processed_data (historic)
- WRITE : *.csv - data/processed_data : concatenation of previous processed data and new processed data
- WRITE : *.json - data/processed_data : wright the IDs stored by websites so that the first script know which data to keep from the fresh scrapped data

In case of new processing to apply to all data follow this procedure, run the new_process.py scriptun `utils`folder, it will :
- Delete processed_data and json IDs  
- Make a concatenation of all historical raw_data and
- set them in the new_tmp_data
- Run this notebook to compute all processing on all data


In [132]:
import os
import json
import pandas as pd
from utils import functions as f

# 1. Loading

In [133]:
FOLDER = 'data/new_tmp_data'
TITLE = 'new_clean_data.csv'
path = '{}/{}'.format(FOLDER, TITLE)
df = pd.read_csv(path)
df.head()

,prix,surface,prix_m2,ville,code_postal,origine,dept,id_


# 2. Processing

In [134]:
if False:
    # Dealing with Paris
    df['ville'] = df[['ville', 'code_postal']].apply(paris_decompo, axis=1)

In [135]:
if False:
    # Dealing with viager
    df['viager'] = df['contenu'].apply(lambda x: True if 'viager' in x else False)
    df_viager = df[df['viager']]
    df_viager_new = df_viager[df_viager.index>last_analyse]
    print('Nombre d\'appartement en viager au total', df_viager.shape[0])
    print('Nombre d\'appartement en viager nouveau', df_viager_new.shape[0])

In [136]:
df_processed = df.copy()

# 3. Saving

In [137]:
def load_old():
    path = 'data/processed_data/process_data.csv'
    if os.path.isfile(path):
        return pd.read_csv(path)
    print('No old processed data at : \n{}'.format(path))
    return pd.DataFrame()

In [138]:
def save_process_data_history(df):
    folder = 'data/processed_data/history'
    now = f.get_now()
    path = '{}/processed_{}.csv'.format(folder, now)
    df.to_csv(path, header=True, index=False)
    return df

In [139]:
def concat_process_with_previous(df_new, df_old):
    folder = 'data/processed_data'
    path = '{}/process_data.csv'.format(folder)
    df_concat = (pd.concat([df_old, df_new], sort=True)
                 .drop_duplicates()
                 .reset_index(drop=True))
    df_concat.to_csv(path, index=False)
    return df_concat

In [140]:
def save_ids(df_concat):
    for source in ['lbc', 'sl', 'pv']:
        list_lbc = json.loads(df_concat[df_concat['origine']==source]['id_'].to_json())
        with open('data/processed_data/list_{}_id.json'.format(source), 'w') as fp:
            json.dump(list_lbc, fp)
    return df_concat

In [141]:
# Loading old data
df_old = load_old()

# Saving all results
df_concat = (df_processed.pipe(save_process_data_history)
             .pipe(concat_process_with_previous, df_old)
             .pipe(save_ids))

In [ ]:

#save_process_data_history(df_processed)


In [ ]:
#df_concat = concat_process_with_previous(df_new, df_old)
#save_ids(df_concat)